In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_pinecone import PineconeVectorStore
import os
from dotenv import load_dotenv
from pinecone.grpc import PineconeGRPC as Pinecone
from langchain.embeddings import HuggingFaceEmbeddings

/Users/AjaysPC1/Documents/research_paper_RAG_chain/rp_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"""
    Load the environment variables and setup constants
"""
load_dotenv()
PINECONE_API_KEY= os.environ.get('PINECONE_API_KEY2')
GOOGLE_API_KEY= os.environ.get('GOOGLE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "research-paper-llm-db5"
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest",
                               temperature=0.3)

vectorstore = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":5})


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

/var/folders/yr/ccrg_j914ln8wydhc308crs00000gp/T/ipykernel_68680/2408064172.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [3]:
def get_answer(question: str):
    """
    Get the answer to a question using the RAG chain.
    """
    response = rag_chain.invoke({"input": question})
    return response["answer"]